In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
is_mps = torch.backends.mps.is_available()
device = torch.device('mps' if is_mps else 'cpu')
print('current_device : ',device)

current_device :  mps


In [7]:
import os
# os.mkdir('/Volumes/My Passport/dataset/data')
!unzip -qq '/Volumes/My Passport/dataset/dataset.zip' -d '/Volumes/My Passport/dataset/data'

In [41]:
import os
original_dataset_dir = '/Volumes/My Passport/dataset/data'
classes_list = []

for i in os.listdir(original_dataset_dir):
    if os.path.isdir(os.path.join(original_dataset_dir,i)):
        classes_list.append(i)

base_dir = '/Volumes/My Passport/dataset/splitted'
os.mkdir(base_dir)

In [42]:
#데이터 분기할 폴더 트리 생성
import shutil
train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir,'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir,'test')
os.mkdir(test_dir)

for cls in classes_list:
    print(cls)
    os.mkdir(os.path.join(train_dir,cls))
    os.mkdir(os.path.join(validation_dir,cls))
    os.mkdir(os.path.join(test_dir,cls))

Pepper,_bell___healthy
Grape___Esca_(Black_Measles)
Pepper,_bell___Bacterial_spot
Strawberry___healthy
Grape___Black_rot
Corn___Common_rust
Apple___Apple_scab
Potato___healthy
Potato___Late_blight
Cherry___healthy
Tomato___Bacterial_spot
Apple___Black_rot
Cherry___Powdery_mildew
Corn___Cercospora_leaf_spot Gray_leaf_spot
Peach___healthy
Tomato___Early_blight
Tomato___Tomato_Yellow_Leaf_Curl_Virus
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Potato___Early_blight
Grape___healthy
Apple___Cedar_apple_rust
Corn___Northern_Leaf_Blight
Tomato___Septoria_leaf_spot
Corn___healthy
Strawberry___Leaf_scorch
Tomato___Tomato_mosaic_virus
Tomato___Leaf_Mold
Tomato___Late_blight
Tomato___healthy
Tomato___Spider_mites Two-spotted_spider_mite
Apple___healthy
Tomato___Target_Spot
Peach___Bacterial_spot


In [61]:
#데이터 현황 확인
import math

for cls in classes_list:
    if os.path.isdir(os.path.join(original_dataset_dir, cls)):
        path = os.path.join(original_dataset_dir,cls)
        #DS.store 파일 무시
        fnames = sorted((f for f in os.listdir(path) if not f.startswith('.')),key=str.lower)

        train_size = math.floor(len(fnames) * 0.6)
        validation_size = math.floor(len(fnames) * 0.2)
        test_size = math.floor(len(fnames) * 0.2)
        
        train_fnames = fnames[:train_size]
        print("Train size(",cls,") ", len(train_fnames))
        for fname in fnames:
            source = os.path.join(path,fname)
            destination = os.path.join(os.path.join(train_dir,cls),fname)
            shutil.copyfile(source,destination)
        
        validation_fnames = fnames[train_size:(validation_size + test_size)]
        print("Train size(",cls,") ", len(train_fnames))
        for fname in fnames:
            source = os.path.join(path,fname)
            destination = os.path.join(os.path.join(train_dir,cls),fname)
            shutil.copyfile(source,destination)
        
        test_fnames = fnames[:train_size]
        print("Train size(",cls,") ", len(train_fnames))
        for fname in fnames:
            source = os.path.join(path,fname)
            destination = os.path.join(os.path.join(train_dir,cls),fname)
            shutil.copyfile(source,destination)
        
        

Train size( Pepper,_bell___healthy )  886
Train size( Grape___Esca_(Black_Measles) )  829
Train size( Pepper,_bell___Bacterial_spot )  598
Train size( Strawberry___healthy )  273
Train size( Grape___Black_rot )  708
Train size( Corn___Common_rust )  715
Train size( Apple___Apple_scab )  378
Train size( Potato___healthy )  91
Train size( Potato___Late_blight )  600
Train size( Cherry___healthy )  512
Train size( Tomato___Bacterial_spot )  1276
Train size( Apple___Black_rot )  372
Train size( Cherry___Powdery_mildew )  631
Train size( Corn___Cercospora_leaf_spot Gray_leaf_spot )  307
Train size( Peach___healthy )  216
Train size( Tomato___Early_blight )  600
Train size( Tomato___Tomato_Yellow_Leaf_Curl_Virus )  3214
Train size( Grape___Leaf_blight_(Isariopsis_Leaf_Spot) )  645
Train size( Potato___Early_blight )  600
Train size( Grape___healthy )  253
Train size( Apple___Cedar_apple_rust )  165
Train size( Corn___Northern_Leaf_Blight )  591
Train size( Tomato___Septoria_leaf_spot )  1062